In [1]:
import gurobipy as grb
import numpy as np
import itertools
import sys
import copy
import collections
import functools

In [2]:
def iterer(*args):
    return itertools.product(*[x_ if isinstance(x_,collections.Iterable) else range(x_) for x_ in args])

In [3]:
possible_plans = [
        ['orange','black','green'],
        ['blue','black','yellow'],
        ['blue','green','orange'],
        ['yellow','green','black'],
        ['black','yellow','orange'],
        ['green','yellow','blue'],
        ['blue','orange','black'],
        ['green','orange','yellow'],
        ['black','blue','green'],
        ['orange','blue','yellow']
    ]

colormap = {
    'black'  : 0,
    'green'  : 1,
    'blue'   : 2,
    'orange' : 3,
    'yellow' : 4
}

In [4]:
"""
    places in heaps: (1 is closer to orange side, 0 to central tracking device)
       | 1|
    | 0| 4| 2|
       | 3|

    colors of cubes: (same as for orange side)
    
    black  -- 0
    green  -- 1
    blue   -- 2
    orange -- 3
    yellow -- 4
    
    numbers of heaps - 0,1,2,3,4,5 CCW starting from closest heap to central tracking device on the orange side
    
    man = 0, 1, 2 (picking 1 4 3 if from 0 for orange!)
    
    cubes:
    -1 - none
    
""";


In [5]:
class STNode():
    def __init__(self, state, upper):
        self.state = state
        self.upper = upper
        self.nexts = []
        
    def __str__(self):
        return str(self.state)
    
    def gen_reversed_path(self):
        sn = self
        while sn != None:
            yield sn
            sn = sn.upper
    
    def get_path(self):
        return reversed(list(self.gen_reversed_path()))

In [6]:
orange_heap = np.array([
                        [-1, 0,-1],
                        [ 1, 4, 3],
                        [-1, 2,-1]
                    ],dtype=np.float)
green_heap  = np.array([
                        [-1, 0,-1],
                        [ 3, 4, 1],
                        [-1, 2,-1]
                 ],dtype=np.float)
orange_heap[orange_heap == -1] = np.nan
green_heap[green_heap == -1] = np.nan
def get_heap(heap_num, side):
    heap = None
    if heap_num in [0,1,2]:
        heap = orange_heap
    if heap_num in [3,4,5]:
        heap = green_heap
    return np.rot90(heap,side)

In [155]:
class CubesState():
    initial_cubes = np.ones((3,5),dtype=np.int)*np.nan
    magic_color = 7
    def __init__(self, heap_num, cubes = None, **kvargs):
        self.heap_num = heap_num
        self.heap = get_heap(heap_num,0)
        self.cubes = cubes if cubes else copy.deepcopy(self.initial_cubes)
        self.vars = kvargs
        self.n_picks = 0
        self.time = 0
        self.x = np.zeros((2,),dtype=np.int)
        self.a = 0
        self.params = kvargs
        self.set_defaults()
    
    def set_defaults(self):
        pass
    
    def remove_color(self, color):
        self.heap[self.heap == color] = np.nan
        
    def cubes_left(self):
        return np.sum(self.heap == self.heap)
    
    def get_cubes_coords(self):
        for i,j in iterer(3,3):
            if self.heap[i][j] == self.heap[i][j]:
                yield i,j
    
    def pick_cube_by(self, color, man, remove=True):
        if remove:
            self.remove_color(color)
        self.cubes[man] = np.roll(self.cubes[man],1)
        self.cubes[man][0] = int(color)
    
    def has_free_space(self,man):
        return not np.all(self.cubes[man] == self.cubes[man])
    
    def pick_cubes(self,colors,mans):
        for c,m in zip(colors, mans):
            self.pick_cube_by(c,m)
    
    def test_pick_cubes(self, colors, mans):
        for c, m in zip(colors,mans):
            if not self.has_free_space(m) or c != c:
                return False
        return True
    
    def is_full(self):
        return np.all(self.cubes == self.cubes)
    
    @staticmethod
    def equal_to_plan(subs, plan):
        tp = copy.deepcopy(subs)
        tp2 = np.array(list(reversed(subs)))
        for i, c in enumerate(tp):
            if c == self.magic_color:
                tp[i] = plan[i]
        for i, c in enumerate(tp2):
            if c == self.magic_color:
                tp2[i] = plan[i]
        return np.all(tp==plan) or np.all(tp2==plan)
    
    def count_plans(self, plan):
        rev_plan = np.array(list(reversed(plan)))
        has_plan = np.zeros((3,), dtype=np.bool)
        for man, s in iterer(3,3):
            has_plan[man] = has_plan[man] or self.equal_to_plan(self.cubes[man][s:s+3],plan)
        return np.sum(has_plan)
    
    @staticmethod
    def sum_cubes(c1,c2):
        h1 = (c1 == c1).argmin(axis=1) + 5*(not np.any(c1 != c1))
        h2 = (c2 == c2).argmin(axis=1) + 5*(not np.any(c2 != c2))
        if np.any(h1 + h2 > 5):
            return None
        c = c1 + 0
        for m in range(3):
            c[m][h1[m]:h2[m]+h1[m]] = c2[m][:h2[m]]
    
    def sum_with_other_state(self, s):
        ns = copy.deepcopy(self)
        for man in range(3):
            for c in reversed(s.cubes[man]):
                if c == c and self.has_free_space(man):
                    ns.pick_cube_by(c,man,False)
        ns.time = s.time + self.time
        return ns
    
    def __str__(self):
        s = "---STATE---" + '\n'
        for l in self.heap:
            s += str(l) + '\n'
        s += "x: %d\ty: %d\ta: %d\n"%(self.x[0], self.x[1], self.a)
        #for k,v in self.vars.items():
        #    s += "%s : %s\t"%(str(k),str(v))
        for m in self.cubes:
            for c in m:
                if c == c:
                    s += "%d "%c
            s += '\n'
        s += '-----------'
        return s
    
    def next_picks(self):
        pass
    
    def next_moves(self):
        pass

In [156]:
class MainRobotState(CubesState):
    defaults = {
            'move_time'  :1,
            'pick_time'  :1,
            'rotate_time':1
        }
    banned_x = {'0' : {},
                '1' : {},
                '2' : {},
                '3' : {},
                '4' : {},
                '5' : {}
    }
    shift = np.array([[-1,0],[0,-1],[1,0],[0,1]])
    mans_shift = np.array([-1,0,1])
    manipulator_masks = [list(x) for x in itertools.product([0,1], repeat=3)][1:]
    def find_a_by_shift(self, sh):
        #print(self.shift, sh)
        return np.where((self.shift == sh).all(axis=1))[0][0]
    
    def set_defaults(self):
        for k,v in self.defaults.items():
            if k not in self.params:
                self.params[k] = v
    
    def new_state_x(self, x, a):
        if self.allowed_position(x,a):
            s = copy.deepcopy(self)
            s.a = a
            s.x = x
            s.time += self.get_time_between(s)
            yield s
    
    def new_state_p(self, mans):
        s = copy.deepcopy(self)
        s.time += self.params['pick_time']
        ms = [i for i, m in enumerate(mans) if m]
        x = self.get_mans_x()
        colors = [self.heap[tuple(x[m])] for m in ms]
        s.pick_cubes(colors, ms)
        return s
    
    def get_mans_x(self):
        x = np.zeros((3,2),dtype=np.int)
        x += self.x + 1
        a_index = (self.mans_shift + self.a) % 4
        # print(a_index)
        return x + self.shift[a_index]
    
    def check_mans_x(self):
        X = self.get_mans_x()
        R =[np.all(np.logical_and(x >= 0, x < 3)) and self.heap[tuple(x)] == self.heap[tuple(x)] and self.has_free_space(i) for i,x in enumerate(X)]
        return R
        
    def get_time_between(self, s):
        return np.max(np.abs(self.x - s.x))*self.params['move_time'] + \
                np.min([np.abs(self.a - s.a), np.abs(self.a - s.a - 4), np.abs(self.a - s.a + 4)])*self.params['rotate_time']
    
    def allowed_position(self,x,a):
        return True
    
    def next_moves(self):
        cubes_left = self.cubes_left()
        if cubes_left in [3,4,5]:
            for a in range(4):
                yield from self.new_state_x(self.x, a)
        elif cubes_left == 2:
            x = np.array(list([i,j] for i,j in self.get_cubes_coords()))
            rest_cube = x.sum(axis=0)
            a0 = self.find_a_by_shift(rest_cube - 2)
            for da in range(-1,2):
                a = (a0+da)%4
                yield from self.new_state_x(self.x, a)
            for da in range(-1,2):
                a = (4-a0+da)%4
                yield from self.new_state_x(rest_cube[0], a)
        else:
            for a,da in iterer(3,range(-1,2)):
                yield from self.new_state_x(-self.shift[(a+da)%4], a)
                
    def next_picks(self):
        mans_can_pick = self.check_mans_x()
        # print(mans_can_pick)
        for mans in self.manipulator_masks:
            # print(np.logical_or(mans_can_pick, np.logical_not(mans)))
            if np.all(np.logical_or(mans_can_pick, np.logical_not(mans))): 
                yield self.new_state_p(mans)
                
    def new_states(self):
        for s in self.next_moves():
            yield from s.next_picks()

In [163]:
class PickQueryOptimizer():
    defaults = {
        'plan': [0,1,2],
        'heaps': [0,1,2],
        
    }
    def __init__(self, StateClass, initial_cubes = None, **kvargs):
        self.StateClass = StateClass
        self.initial_cubes = initial_cubes
        self.params = {}
        for k,v in self.defaults.items():
            self.params[k] = copy.copy(v)
            
        for k,v in kvargs.items():
            if k in self.params.keys():
                self.params[k] = copy.copy(v)
                
    def set_plan(self, plan):
        if isinstance(plan[0], str):
            self.params['plan'] = [colormap[c] for c in plan]
        else:
            self.params['plan'] = plan
        
    def get_plan(self, num = None):
        if num == None:
            return self.params['plan']
        elif num in [0,1,2]:
            return self.params['plan'][num]
        return self.params['plan']
    
    def checker_node_for_plans(self, x):
        def f(sn):
            return sn.state.count_plans() == x
        return f
    
    def reduce_by_colors(self, final_states):
        extra_colors = set([0,1,2,3,4]) - set(self.get_plan())
        cubes = {}
        for sn in final_states:
            s = sn.state
            tokey = copy.deepcopy(s.cubes)
            for ec in extra_colors:
                tokey[tokey == ec] = 6
            key = str(tokey)
            if key in cubes:
                if cubes[key].state.time > s.time:
                    cubes[key] = sn
            else:
                cubes[key] = sn
        return list(cubes.values())
    
    def build_solution_tree(self, heap_num, use_initial_cubes=True, only_good=False):
        initial_state = self.StateClass(heap_num, self.initial_cubes if use_initial_cubes else None)
        root = STNode(initial_state, None)
        this_wave = [root]
        next_wave = []
        N = 0
        final_states = []
        while len(this_wave) > 0:
            for node in this_wave:
                if node.state.cubes_left() == 0:
                    final_states.append(node)
                node.nexts = [STNode(x,node) for x in node.state.new_states()]
                next_wave.extend(node.nexts)
            if len(next_wave) > 0:
                N = len(next_wave)
            this_wave = next_wave
            next_wave = []
        
        print(len(final_states))
        # fss = self.reduce_final_states(final_states)
        fss2 = self.reduce_by_colors(final_states)
        print(len(fss2))
        if only_good:
            return root, list(filter(self.checker_node_for_plans(1), fss2))
        else:
            return root, fss2
    
    def save(self, heaps=None, only_good=False):
        heaps = heaps if heaps else self.params['heaps']
        self.fs = [self.build_solution_tree(hn, i==0,only_good)[1] for i,hn in enumerate(heaps)]
        
    
    def count_final_states(self, heaps=None, only_good=False):
        c = [[s.cubes for s in f] for f in self.fs]
        for f1,f2 in iterer(fs[0],fs[1]):
            s1 = f1.state
            s2 = f2.state
            sf = s1.sum_with_other_state(s2)
            key = str(sf.cubes)
            if key in cubes2:
                if cubes2[key].time > sf.time:
                    cubes2[key] = sf
            else:
                cubes2[key] = sf
            
        print(len(cubes2.values()))
            
    

In [164]:
pqo = PickQueryOptimizer(MainRobotState)

In [165]:
pqo.save()

1791
144
1791
144
1791
144


In [158]:
fs = pqo.fs

AttributeError: 'PickQueryOptimizer' object has no attribute 'fs'

In [138]:
s = MainRobotState(0)
print(s)

---STATE---
[nan nan nan]
[1. 4. 3.]
[nan  2. nan]
x: 0	y: 0	a: 0



-----------


In [94]:
for ns in s.get_cubes_coords():
    print(ns)

(0, 1)
(1, 0)
(1, 1)
(1, 2)
(2, 1)


In [75]:
a = np.array([[1,2],[3,0]])

In [80]:
a.sum(axis=0) - 1

array([3, 1])

In [139]:
s.pick_cube_by(0,1)

In [140]:
s.cubes

array([[nan, nan, nan, nan, nan],
       [ 0., nan, nan, nan, nan],
       [nan, nan, nan, nan, nan]])

In [145]:
z= (s.cubes == s.cubes).argmin(axis=1)

In [154]:
s.cubes[1,0:1]

array([0.])